<a href="https://colab.research.google.com/github/riccardocorradi/Trading-Room-Workshop/blob/main/Client_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final client code, has rebalancing. Need to add a minimum threshold to trigger the rebalancing, or it may happen to rebalance back due to market impact.



Below is likely the FINAL of my updates, should be faster and more efficient, also updated logic for if only some of our orders get filled. I deleted old versions of my updates from here to keep it simple but I did save one locally if we need to refer back to it

In [ ]:
# STATE OF THE ART CLIENT CODE FINAL

import socket
import threading
import requests
import numpy as np
from time import sleep

TRCOSTPERSHARE = 0.02                               # pre-set trading costs per share
REBALANCING_SPEEDBUMP = 0.1                         # speedbump to reduce rebalancing market impact
TOLERANCE = 0.2                                     # rebalancing tolerance per share
TOLERANCE_GEM_TOWARD = 1
TOLERANCE_GEM_AWAY = 0.3

# Custom exception for API errors
class ApiException(Exception):
    pass

# API key for authorization
API_KEY = {'X-API-Key': 'ABC123'}
Q = 5000

# Function to get the current tick from the server
def get_tick(session):
    resp = session.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick']
    raise ApiException('Invalid API key provided.')

# Function to get bid/ask prices, returns bid_price, ask_price
def ticker_bid_ask(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities/book', params=payload)
    if resp.ok:
        book = resp.json()
        return book['bids'][0]['price'], book['asks'][0]['price']
    raise ApiException('Invalid API key provided.')

# Function to fetch news updates from the server for a given ticket
def get_news(s):
    payload = {'since': 200}
    resp = s.get('http://localhost:9999/v1/news', params=payload)
    if resp.ok:
        return resp.json()  # Return news data if successful
    return []  # Return empty list if not successful

# Function to extract estimates from the news data
def ext_est(news):
    estimates = {}
    for mess in news:
        if mess['news_id'] != 1:  # Ignore old news
            time = mess['tick']
            text = mess['body']
            dollarpos = text.find("$")  # Find dollar sign in the message
            estimate = "".join(list(text)[dollarpos + 1:])  # Extract estimate
            estimates[time] = float(estimate)  # Store estimate with its tick
    return estimates

# Get gross position
def get_position(session):
    resp = session.get('http://localhost:9999/v1/limits')
    if resp.ok:
        data = resp.json()
        position = data[0]["gross"]
    return position

# Function to get current exposure (position) for a security
def get_exposure(session, ticker):
    payload = {'ticker': ticker}
    resp = session.get('http://localhost:9999/v1/securities', params=payload)
    if resp.ok:
        positiondata = resp.json()
        return positiondata[0]['position']
    raise ApiException('Invalid API key provided.')

# Function to get expected profit
def expected_profit(s, ticker, lowerbound, upperbound):
    bid, ask = ticker_bid_ask(s, ticker)
    if ask + TRCOSTPERSHARE < lowerbound:
        profit = lowerbound - ask - TRCOSTPERSHARE
    elif bid - TRCOSTPERSHARE > upperbound:
        profit = bid - upperbound - TRCOSTPERSHARE
    else:
        profit = 0
    return profit

# Function to place a limit order
def place_order(s, ticker, order_type, quantity, action, price):
   while True:
      try:
          # Send a POST request to place an order
          response = s.post('http://localhost:9999/v1/orders', params={
              'ticker': ticker,
              'type': order_type,
              'quantity': quantity,
              'action': action,
              'price': price
          })

          # Check the response status
          if response.status_code == 200:
              order_info = response.json()
              print(f"Order placed successfully: {order_info}")
              break

          elif response.status_code == 429:
              wait_time = response.json().get('wait', 0)
              print(f"Rate limit exceeded. Waiting {wait_time} seconds before retrying.")
              sleep(wait_time)
              continue

          else:
              print(f"Failed to place order: {response.status_code}, {response.json()}")
              break

      except Exception as e:
          print(f"Error placing order: {e}")
          break

# Function to reset limit orders on current bounds of a security
def limit_on_bounds(s, security, security_ub, security_lb):
      print("Security:", security, " Upperbound:", security_ub, " Lowerbound:", security_lb)
      curr_position = get_exposure(s, security)
      print("net position is", curr_position, "on", security)

      if curr_position > 0:                        # cancel and re-set limit orders at the new bounds on currently traded security
          s.post("http://localhost:9999/v1/commands/cancel", params={'all': 1, 'ticker': security})
          for i in range (0,40):
              place_order(s, security, "LIMIT", Q, "SELL", security_ub)

          curr_position = get_exposure(s, security)
          num_orders = 20 - np.ceil(abs(curr_position / Q))
          if curr_position < 100000:
              i = 0
              while i < num_orders and num_orders >= 1:
                  try:
                    place_order(s, security, "LIMIT", Q, "BUY", security_lb)
                    i = i + 1
                  except Exception as e:
                      print(e)
                      break
              curr_position = abs(get_exposure(s, security))               # set limit order for whatever remains, if any
              if curr_position != 100000:
                  order_size = min(100000 - curr_position, Q)
                  place_order(s, security, "LIMIT", order_size, "BUY", security_lb)

      if curr_position < 0:
          s.post("http://localhost:9999/v1/commands/cancel", params={'all': 1, 'ticker': security})
          for i in range (0,40):
              place_order(s, security, "LIMIT", Q, "BUY", security_lb)

          curr_position = get_exposure(s, security)
          num_orders = 20 - np.ceil(abs(curr_position / Q))
          if curr_position > -100000:
              i = 0
              while i < num_orders and num_orders >=1:
                  try:
                    place_order(s, security, "LIMIT", Q, "SELL", security_ub)
                    i = i + 1
                  except Exception as e:
                      print(e)
                      break
              curr_position = abs(get_exposure(s, security))               # set limit order for whatever remains, if any
              if curr_position != 100000:
                  order_size = min(100000 - curr_position, Q)
                  place_order(s, security, "LIMIT", order_size, "SELL", security_ub)


def receive_messages(client_socket, s):               # function that does both receiving information and placing orders
    ub_lowerbound = 40                                # initialization of relevant parameters
    ub_upperbound = 60
    gem_lowerbound = 20
    gem_upperbound = 30
    etf_lowerbound = 60
    etf_upperbound = 90

    while True:                                                             # main loop where the magic happens
        try:
            response = client_socket.recv(1024).decode()                    # fetch the server's string of messages separated by _
            if response:
                print("Received from server:",response)
                messagelist = response.split("_")                           # split the string into messages by recognizing _ and add all messages to a message list

                for i in messagelist:                                       # for each message in the message list, do the following:
                    try:
                        print(i)                                            # Debug: print the message
                        message_ticker = i.split()[0]                       # get the message's ticker
                        message_lb = float(i.split()[4].replace("$",""))    # get the message's lowerbound
                        message_ub = float(i.split()[6].replace("$",""))    # get the message's upperbound
                    except:
                        continue

                    message_lb = min(message_lb, message_ub)                # correction in case the server gets a lowerbound > upperbound
                    message_ub = max(message_lb, message_ub)

                    if message_ticker == "UB":                              # update each security's bounds if the message concerns that security
                        ub_lowerbound = message_lb
                        ub_upperbound = message_ub
                        if ub_lowerbound <= 40:
                          ub_lowerbound = 40.01
                        if ub_upperbound >= 60:
                          ub_upperbound = 59.99
                                                                            # and ensure UB and GEM bounds are within case limits
                    elif message_ticker == "GEM":
                        gem_lowerbound = message_lb
                        gem_upperbound = message_ub
                        if gem_lowerbound <= 20:
                          gem_lowerbound = 20.01
                        if gem_upperbound >= 30:
                          gem_upperbound = 29.99

                    elif message_ticker == "ETF":
                        etf_lowerbound = message_lb
                        etf_upperbound = message_ub

            exp_profit_ub = expected_profit(s,"UB", ub_lowerbound, ub_upperbound)               #compute the expected profits of each security's trade strategy
            exp_profit_gem = expected_profit(s, "GEM", gem_lowerbound, gem_upperbound)
            exp_profit_etf = expected_profit(s, "ETF", etf_lowerbound, etf_upperbound)

            print("Expected profit for UB is:",exp_profit_ub)                                   #debug: Print the expected profits
            print("Expected profit for GEM is:", exp_profit_gem)
            print("Expected profit for ETF is:", exp_profit_etf)

            exp_profits = abs(np.array([exp_profit_ub, exp_profit_gem, exp_profit_etf]))       # build an array of all exp. profits
            securities = ["UB", "GEM", "ETF"]
            bounds = [[ub_lowerbound, ub_upperbound], [gem_lowerbound, gem_upperbound], [etf_lowerbound, etf_upperbound]]  # build a lists of bound lists
            maximiser_index = np.argmax(exp_profits)                                           # find the index of the maximum in the exp_profits vector
            security = securities[maximiser_index]                                             # get the name of the profit maximising security

            if exp_profits[maximiser_index] == 0:                                              # check if there are any opportunities at all
                continue

            elif get_position(s) != 0:                                                         # check if we have a current position

                # check if substituting the security would make sense

                # get exposures of all securities
                exposures = {ticker: abs(get_exposure(s, ticker)) for ticker in ["UB", "GEM", "ETF"]}

                # find the security with the largest exposure and label it old_security
                old_security = max(exposures, key=exposures.get)

                # clear positions for all other securities that are not old_security
                for ticker, exposure in exposures.items():
                    if ticker != old_security and exposure > 0:
                        direction = "BUY" if get_exposure(s, ticker) < 0 else "SELL"
                        print(f"Clearing position for {ticker}, exposure: {exposure}, direction: {direction}")

                        n = 0
                        while abs(get_exposure(s, ticker)) > 0:
                            current_exposure = abs(get_exposure(s, ticker))
                            order_size = min(current_exposure, Q)  # place orders of size Q or whatever remains
                            if order_size <= 0:
                                break
                            try:
                                place_order(s, ticker, "MARKET", order_size, direction, "")
                                sleep(REBALANCING_SPEEDBUMP)
                            except Exception as e:
                                print(f"Error while clearing position for {ticker}: {e}")
                                break
                            n = n + 1
                            if n > 20:                                                                # prevent unlimited loop
                                break

                # get current expected profit of new position
                new_security = securities[maximiser_index]
                new_security_lb, new_security_ub = bounds[maximiser_index][0], bounds[maximiser_index][1]
                new_security_expected_profit = exp_profits[maximiser_index]

                if new_security == old_security:                                              # if the current maximiser is the same one you're trading, don't do anything and continue updating info
                    limit_on_bounds(s, new_security, new_security_ub, new_security_lb)        # cancel and re-set limit orders at the new bounds on currently traded security
                    continue

                else:

                    old_security_index = securities.index(old_security)                       # fetch the index of the old security (0,1,2)
                    old_security_lb, old_security_ub = bounds[old_security_index][0], bounds[old_security_index][1]   #get its current bounds

                    old_security_bid, old_security_ask = ticker_bid_ask(s, old_security)      # get bids and asks for each security, old and new
                    new_security_bid, new_security_ask = ticker_bid_ask(s, new_security)

                    curr_exp_profit_old_security = exp_profits[old_security_index]            # get the current exp. profit of the maximiser you traded before


                    print(new_security, "offers profit of", new_security_expected_profit)
                    print(old_security, "offers profit of", curr_exp_profit_old_security)


                    target_gross_profit = (new_security_expected_profit - curr_exp_profit_old_security)     # compute the profit of entering the new position, gross of tr_cost

                    if old_security == "GEM":
                        current_tol = TOLERANCE_GEM_AWAY
                    elif new_security == "GEM":
                        current_tol = TOLERANCE_GEM_TOWARD
                    else:
                        current_tol = TOLERANCE

                    if target_gross_profit > current_tol:                                # only rebalance if the new profit > old profit + tolerance per share
                        print("Rebalancing triggered")
                        print("Would substitute", old_security, "with", new_security, "to gain", target_gross_profit - TRCOSTPERSHARE)

                        if get_exposure(s, old_security) < 0:
                            unload_direction = "BUY"
                        else:
                            unload_direction = "SELL"

                        if new_security_ask < new_security_lb:
                            reload_direction = "BUY"
                        if new_security_bid > new_security_ub:
                            reload_direction = "SELL"

                        s.post("http://localhost:9999/v1/commands/cancel", params={'all': 1, 'ticker': old_security})  # cancel all outstanding limit orders on old security


                        old_position = abs(get_exposure(s, old_security))
                        new_position = abs(get_exposure(s, new_security))

                        # alternately unload old position and reload new position
                        n = 0
                        while old_position > 0 or new_position < 100000:
                            try:
                                if old_position > 0:
                                    order_size = min(old_position, Q)
                                    if order_size <= 0:
                                        break
                                    if order_size > 0:
                                        place_order(s, old_security, "MARKET", order_size, unload_direction, "")
                                        print(f"Tried to {unload_direction} {order_size} of {old_security}")
                                        sleep(REBALANCING_SPEEDBUMP)
                                    # update exposure after the order
                                    old_position = abs(get_exposure(s, old_security))

                                # reload new security in chunks of Q
                                if new_position < 100000:
                                    remaining_exposure = 100000 - new_position
                                    order_size = min(remaining_exposure, Q)
                                    if order_size <= 0:
                                        break
                                    if order_size > 0:
                                        place_order(s, new_security, "MARKET", order_size, reload_direction, "")
                                        print(f"Tried to {reload_direction} {order_size} of {new_security}")
                                        sleep(REBALANCING_SPEEDBUMP)
                                    # update exposure after the order
                                    new_position = abs(get_exposure(s, new_security))
                                n = n + 1
                                if n > 20:                                                                               # prevent unlimited loop, 20 orders is max
                                    break

                            except Exception as e:
                                print(f"Error while rebalancing between {old_security} and {new_security}: {e}")
                                break


                        if reload_direction == "BUY":
                          opp_direction = "SELL"
                          opp_bound = new_security_ub
                        else:
                          opp_direction = "BUY"
                          opp_bound = new_security_lb
                                                                                                          # place limit orders on opposite bound once fully rebalanced
                        for i in range (0,40):
                          place_order(s, new_security, "LIMIT", Q, opp_direction, opp_bound)

                    else:
                        limit_on_bounds(s, old_security, old_security_ub, old_security_lb)                   # if rebalancing not worth it, replace limit orders on old security bounds
                        continue

            else:       # build a 100k position on the maximizer security
                security_bid, security_ask = ticker_bid_ask(s, security)                                    # get bids/asks for the maximizer security
                security_lb, security_ub = bounds[maximiser_index][0], bounds[maximiser_index][1]           # get bounds for the maximizer security

                if security == "GEM":
                    security_lb = security_lb - TOLERANCE
                    security_ub = security_ub + TOLERANCE

                if security_ask < security_lb:                                                              # if the security is underpriced
                    s.post("http://localhost:9999/v1/commands/cancel", params={'all': 1, 'ticker': security})    # kill any outstanding orders

                    for i in range(0, 20):                                                                  # submit buy orders 20 times and sell orders 40 times for 5.000 each
                        try:
                            place_order(s, security, "LIMIT", Q, "BUY", security_lb)
                        except Exception as e:
                            print(e)
                            break

                    for i in range(0, 40):
                        try:
                            place_order(s, security, "LIMIT", Q, "SELL", security_ub)
                        except Exception as e:
                            print(e)
                            break

                elif security_bid > security_ub:                                                            # if the security is overpriced
                    s.post("http://localhost:9999/v1/commands/cancel", params={'all': 1, 'ticker': security})   # kill any outstanding orders

                    for i in range(0, 20):
                        try:                                                                               # submit buy orders 40 times and sell orders 20 times for 5.000 each
                            place_order(s, security, "LIMIT", Q, "SELL", security_ub)

                        except Exception as e:
                            print(e)
                            break

                    for i in range(0, 40):
                        try:
                            place_order(s, security, "LIMIT", Q, "BUY", security_lb)

                        except Exception as e:
                            print(e)
                            break

        except Exception as e:
            print(f"Connection error: {e}")
            break

def send_estimates(client_socket, s):
    curr_news_UB = []                                     # Initialize current news data for UB
    curr_news_GEM = []                                    # Initialize current news data for GEM
    curr_news_all = []

    while True:                                           # Main loop to continuously fetch news and send estimates

        new_news_all = get_news(s)                        # fetch all news
        if new_news_all != curr_news_all:                 # if they're different from current news (meaning there are some new news)
            curr_news_all = new_news_all                  # update the current news to the new ones
            for item in curr_news_all:                    # for every piece of news in the current (updated) ones
                if "UB" in item['headline'].split():      # assign it to the respective news list, depending on the ticker
                    curr_news_UB.append(item)
                if "GEM" in item['headline'].split():
                    curr_news_GEM.append(item)

            ub_estimates = ext_est(curr_news_UB)          # extract estimates for each security from all news items concerning each security
            gem_estimates = ext_est(curr_news_GEM)

             # Process and send estimates for UB
            try:
                latest_ub_tick = max(list(ub_estimates.keys()))         # Get the latest tick
                latest_ub_value = ub_estimates[latest_ub_tick]          # Get the corresponding estimate
                ub_message = f"UB;{latest_ub_tick}-{latest_ub_value}"   # Create the message for UB with the ticker
                client_socket.send(ub_message.encode())                 # Send the message to the server
                print("Sent UB message to server:", ub_message)         # Debug: Confirm UB message sent
            except Exception as e:
                print("Failed to send info for UB:", e)

            # Process and send estimates for GEM
            try:
                latest_gem_tick = max(list(gem_estimates.keys()))          # Get the first tick
                latest_gem_value = gem_estimates[latest_gem_tick]          # Get the corresponding estimate
                gem_message = f"GEM;{latest_gem_tick}-{latest_gem_value}"  # Create the message for GEM with the ticker
                client_socket.send(gem_message.encode())                   # Send the message to the server
                print("Sent GEM message to server:", gem_message)          # Debug: Confirm GEM message sent
            except Exception as e:
                print("Failed to send info for GEM:", e)

            sleep(0.05)

# Main function to start the client
def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)   # Create a socket
    host = '175.159.205.200'                                            # Server IP address
    port = 11111                                                        # Server port number

    client_socket.connect((host, port))                                 # Connect to the server
    print("Connected to the server")

    with requests.Session() as s:                                       # Start a session for API requests
        s.headers.update(API_KEY)                                       # Set the API key in the headers

        # Start one thread to listen for messages from the server for each ticker
        receive_thread = threading.Thread(target=receive_messages, args=(client_socket, s))
        # Start second thread for sending estimates separately
        send_thread = threading.Thread(target=send_estimates, args=(client_socket, s))

        receive_thread.start()
        send_thread.start()

start_client()